In [7]:
from publicsuffixlist import PublicSuffixList
from publicsuffix2 import get_public_suffix, fetch
from dnspy.dnspy import Dnspy
import time, re, json
import pandas as pd

psl = PublicSuffixList()
psl_file = fetch()
d = Dnspy()

regex = {}

dicter_ = {}
with open('dicter.com.json', 'r') as f:
    dicter_['com'] = json.load(f)
with open('dicter.uk.json', 'r') as f:
    dicter_['uk'] = json.load(f)
with open('dicter.all.json', 'r') as f:
    dicterall = json.load(f)


In [101]:
def _dicter(dn):
    ds = []
    idn = '.'.join(dn.split('.')[::-1])
    labels = idn.split('.')
    d = dicterall[idn[0]]
    labels = idn.split('.')
    for i, l in enumerate(labels):
        try:
            d = d[l]
            if '' in d:
                ds.append(d[''])
            elif '@code' in d:
                ds.append(d)
                break
        except KeyError as e:
            if len(labels)-1 > i and '*' in d:
                ds.append(d['*'])
            break

#     for i, l in enumerate(labels):
#         try:
#             d = d[l]
#             if '' in d:
#                 k = d['']
#                 k['dn-suffix'] = '.'.join(labels[i+1:][::-1])
#                 ds.append(k)
#             elif '@code' in d:
#                 k = d
#                 k['dn-suffix'] = '.'.join(labels[i+1:][::-1])
#                 ds.append(k)
#                 break
#         except KeyError as e:
#             if len(labels)-1 > i and '*' in d:
#                 k = d['*']
#                 k['dn-suffix'] = '.'.join(labels[i+1:][::-1])
#                 ds.append(k)
#             break
        
    return [ d['@suffix'] + '|' + d['@section'] for d in ds ]
#     return [ {k.replace('@', ''): d[k] for d in ds for k in d } ]

# def _match(idn):
#     _tld = dn[dn.rfind('.') + 1:]
#     idn = '.'.join(dn.split('.')[::-1])
#     groups = regex[_tld].match(idn)
#     if groups is None: return None
#     groups = groups.groupdict()
#     return [ g for g in groups if groups[g] is not None ]

def perf(name, dn, f):
    s = time.time()
    a = f(dn)
    e = time.time() - s
    return a, e


dns = [
    'amazon.aaa',
    'ciao.co.uk',
    'ciao.issmarterthanyou.com',
    'ciao.compute-1.amazonaws.com',
]

psls = [ 'publicsuffix2', 'Dnspy', 'PublicSuffixList', 
#         'Regex',
        'Dicter' ]

ffs = {
    'publicsuffix2': lambda dn: get_public_suffix(dn, psl_file),
    'PublicSuffixList': lambda dn: psl.publicsuffix(dn),
    'Dnspy': lambda dn: d.subdoms(dn),
#     'Regex': _match,
    'Dicter': _dicter
}

df = {}
for p in psls:
    if p not in df:
        df[p] = {}
    for dn in dns:
        r, t = perf(p, dn, ffs[p])
        df[p][(dn, 't')] = t
        df[p][(dn, 'r')] = r
        
display(pd.DataFrame.from_dict(df).transpose())


/home/princio/anaconda3/envs/datascience/lib/python3.9/site-packages/publicsuffix2/__init__.py:376: UserWarning: This function returns the private suffix, SLD, or registrable domain. This equivalent to function get_sld(). To get the public suffix itself, use get_tld().
  warnings.warn(


amazon.aaa                    ciao.co.uk  \
                          t                  r          t   
publicsuffix2      0.000075         amazon.aaa    0.00001   
Dnspy              0.000018  [aaa, amazon.aaa]   0.000008   
PublicSuffixList   0.000007                aaa   0.000004   
Dicter             0.000005    [aaa|icann-new]   0.000005   

                                          ciao.issmarterthanyou.com  \
                                        r                         t   
publicsuffix2                  ciao.co.uk                  0.000006   
Dnspy                 [co.uk, ciao.co.uk]                  0.000006   
PublicSuffixList                    co.uk                  0.000004   
Dicter            [uk|icann, co.uk|icann]                  0.000004   

                                                                     \
                                                                  r   
publicsuffix2                             ciao.issmarterthanyou.com   
Dnspy             [issmarterthanyou.com, ciao.issmarterthanyou.com]   
PublicSuffixList                               issmarterthanyou.com   
Dicter             [com|icann, issmarterthanyou.com|private-domain]   

                 ciao.compute-1.amazonaws.com                                  
                                            t                               r  
publicsuffix2                        0.000008    ciao.compute-1.amazonaws.com  
Dnspy                                0.000007  [ciao.compute-1.amazonaws.com]  
PublicSuffixList                     0.000003    ciao.compute-1.amazonaws.com  
Dicter                               0.000004                     [com|icann]

In [18]:
import subprocess

s = time.time()
a = subprocess.call(["perl", "regex.com.perl", idn])
e = time.time() - s

print("%-20s %f %s" % ('perl', e, a))


i01589ged0.p03965ged1.perl                 0.015918 0


In [3]:
import numpy as np

df = pslr.etld.frame.fillna('')



n = df.to_numpy()

df = df.convert_dtypes()

df['suffix'] = df.suffix.astype(np.dtype('U40'))

for i, col in enumerate(df.columns):
    try:
        print(i, col, df[col].str.len().max())
    except:
        print(i, col, df[col].iloc[0])
        pass
v = df.values.tolist()

v

# v = v.astype('U10, U50, U20, U5, U15, ?, ?')

# v.dtype

# v[0]

0 code 10
1 suffix 40
2 tld 18
3 punycode 24
4 type 18
5 origin 5
6 section 14
7 newGLTD False
8 exception False


[['p00000ged2',
  '*.0emm.com',
  'com',
  '',
  'generic',
  'both',
  'private-domain',
  False,
  False],
 ['p00001ccd2',
  '*.advisor.ws',
  'ws',
  '',
  'country-code',
  'both',
  'private-domain',
  False,
  False],
 ['p00002ged2',
  '*.alces.network',
  'network',
  '',
  'generic',
  'both',
  'private-domain',
  False,
  False],
 ['p00003ccd2',
  '*.awdev.ca',
  'ca',
  '',
  'country-code',
  'both',
  'private-domain',
  False,
  False],
 ['p00004ccd2',
  '*.azurecontainer.io',
  'io',
  '',
  'country-code',
  'both',
  'private-domain',
  False,
  False],
 ['p00005ccd3',
  '*.backyards.banzaicloud.io',
  'io',
  '',
  'country-code',
  'both',
  'private-domain',
  False,
  False],
 ['p00006ged2',
  '*.banzai.cloud',
  'cloud',
  '',
  'generic',
  'both',
  'private-domain',
  False,
  False],
 ['i00007ccd1',
  '*.bd',
  'bd',
  '',
  'country-code',
  'both',
  'icann',
  False,
  False],
 ['p00008ged2',
  '*.beget.app',
  'app',
  '',
  'generic',
  'both',
  'private

In [20]:
%timeit v[1000]

24.7 ns ± 0.323 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
